In [1]:
#default_exp modules.pointpillars


# Pointpillars wrapper module
> This is a wrapper for all of the separate pointpillar modules. This includes the featurenet, the backbone,
> the detection head and also the box matching module.

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")

In [3]:
#export
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.ops import nms

from modules.pillarcalc import PillarCalc
from modules.featurenet import FeatureNet
from modules.backbone import Backbone
from modules.head import Head
from modules.boxmatch import BoxMatch
from utils.math.box_ops import convert_boxes_to_2d_corners

logger = logging.getLogger(__name__)

NameError: name 'nn' is not defined

In [ ]:
#export
class PointPillars(nn.Module):

    def __init__(self, conf, train_mode: bool = True):
        """A wrapper which combines all of the individual pointpillar modules.

        :param conf:
        :param training: If used in training or prediction mode.

        :returns: Depending on training or prediction mode
        """
        super(PointPillars, self).__init__()

        self.conf = conf
        self.train_mode = train_mode

        self.pillarcalc = PillarCalc(conf['pillars'])
        self.featurenet = FeatureNet(conf['featurenet'], conf['pillars'])
        self.backbone = Backbone(conf['featurenet'])
        self.head = Head(conf['head'])
        self.boxmatch = BoxMatch(conf['pillars'], conf['head'], train_mode)

    def forward(self,
                 batch: torch.Tensor,
                 ind_batch: torch.Tensor,
                 label_batch: torch.Tensor = None,
                 label_mask: torch.Tensor = None):
        """
        Passes the given parameters forward through the network. Can either be for training or at interference
        time. The pil_batch contains the pillar points and the ind_batch contains the pillar values.
        When in training mode, label_batch is expected containing the labels in a tensor form.

        :param batch:     Tensor(nb_attributes, nbr_pillars w. zero_padding, nb_points_in_pillars)
                          Tensor containing the points sorted into their corresponding pillar
        :param ind_batch: Tensor(nb_pillars, 2)
                          Tensor containing the unique index for each pillar to later identify the position
        :param label_batch: Tensor(batch_size, nb_labels, label_attributes)
                            Only required if in training mode.

        :returns:
        """
        """print(batch.element_size() * batch.nelement())
        print(ind_batch.element_size() * ind_batch.nelement())
        print(label_batch.element_size() * label_batch.nelement())
        print(label_mask.element_size() * label_mask.nelement())"""

        """batch = batch.cuda(non_blocking=True)
        ind_batch = ind_batch.cuda(non_blocking=True)
        label_batch = label_batch.cuda(non_blocking=True)
        label_mask = label_mask.cuda(non_blocking=True)"""

        batch = batch.cuda(non_blocking=True)
        ind_batch = ind_batch.cuda(non_blocking=True)
        label_batch = label_batch.cuda(non_blocking=True)
        if label_mask is not None:
            label_mask = label_mask.cuda(non_blocking=True)

        with torch.no_grad():
            batch = self.pillarcalc(batch, ind_batch)
        batch = self.featurenet(batch, ind_batch)
        batch = self.backbone(batch)
        batch = self.head(batch)
        batch = self.boxmatch(batch, label_batch, label_mask)

        if not self.training:
            batch = self.interference(batch)

        return batch


    def interference(self, batch: torch.Tensor) -> tuple:
        """
        Performs nms on a single point cloud. Eliminates all BBs but the most confident Box with
        IoU over a given threshold.

        :returns: (pred_occ, pred_cls, pred_head, pred_box)
        """
        occ_threshold = 0.6
        iou_threshold = 0.2

        pred_occ, _, _, pred_box = batch
        for i, tens in enumerate(batch[:-1]):
            batch[i] = torch.sigmoid(tens)

         # select boxes above pos threshold
        cond = pred_occ[0] >= occ_threshold
        indices = torch.nonzero(cond, as_tuple=False).squeeze(1)
        for i, tens in enumerate(batch):
            batch[i] = torch.index_select(tens, 1, indices)

        pred_box_corners = convert_boxes_to_2d_corners(batch[3])
        # select with non-maximum-suppression from iou
        indices = nms(pred_box_corners[0], batch[0][0], iou_threshold)
        for i, tens in enumerate(batch):
            batch[i] = torch.index_select(tens, 1, indices)

        #rint(batch[3].shape)
        #print(batch[0], batch[0].shape)

        # remove batch dimension
        return batch[0][0], batch[1][0], batch[2][0], batch[3][0]

In [ ]:
#export
def init_weights(m):
    torch.nn.init.uniform(m.weight)